In [1]:
import pinecone

In [2]:
!pip install --upgrade jupyter ipywidgets



In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [4]:
PINECONE_API_KEY = "4d6fb2ad-d4c6-48df-8346-7cc0b3027f18"
PINECONE_API_ENV = "gcp-starter"

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("E:\question_bot\data/")

Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x247 for key /Subtype
Multiple definitions in dictionary at byte 0x8acf for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x3a6 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x342 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x3a6 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x342 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x3a6 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x342 for key /Subtype
Multiple definit

In [7]:
extracted_data

[Document(metadata={'source': 'E:\\question_bot\\data\\Chatbot_in_business.pdf', 'page': 0}, page_content=' \n \nThis is an open access paper under the license  \n \n I C E M I T  \n1st International Scientific Conference on Economy, Management \nand Informa tion Technologies – ICEMIT 2023  \n \n \nChatbot in business  \n \nMilan Puvačaa\uf02a, Ivan Kunacb \n \na Ofir d.o.o., Osijek, Croatia  \nb A. G. Matoš Elementary School, Tovarnik, Croatia  \n \nA r t i c l e   i n f o   A b s t r a c t  \n \nReview paper  \n \nDOI:  \nhttps://doi.org/10.46793/ICEMIT23.219P       \n \nUDC/ UDK:  \n070.3 :004.8    \nThe cost of today\'s employee education is ex tensive, but also dynamic due to the global \nchallenges of business that encompasses each activity. The special expression of the \naforementioned is outlined in the elements of customer support provided to visitors or \npotential clients through the websites o f companies or other institutions. In order to \nsomewhat bridge the gap of new 

In [8]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 83


In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
import os
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'


In [12]:
embeddings = download_hugging_face_embeddings()

c:\Users\realme\anaconda3\envs\vivek2110\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
!pip install --upgrade pinecone-client


In [16]:
import pinecone
from pinecone import Pinecone,ServerlessSpec

In [17]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="business_bot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [18]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are business"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

AttributeError: type object 'Pinecone' has no attribute 'from_existing_index'

In [22]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [23]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [25]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

ImportError: Could not import `ctransformers` package. Please install it with `pip install ctransformers`

In [ ]:
import pinecone

# Initialize Pinecone with API key and environment
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [28]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-66a1d2ad-58042cef72d14de212c8ddf2;aaf06416-d84a-4e93-a56d-4dd191438db4)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [26]:
import ctransformers


In [27]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

NameError: name 'llm' is not defined